# Meshing a protein using PyGAMer

This tutorial will walk you through the process of using PyGAMer to generate a tetrahedral mesh of the volume surrounding a protein. The protein of interest, PDBID 2JHO, is that of sperm whale myoglobin at 1.4Å resolution.

In [1]:
# Load in PyGAMer and numpy
import pygamer
import numpy as np

In [2]:
# Mesh the protein of interest
mesh = pygamer.readPDB_molsurf('../data/2jho.pdb')

When using PyGAMer it's important to initialize the orientation of the simplices. Notably `compute_orientation()` will try to apply a self consistent set of orientations. In many cases, you may wish to ensure that the mesh normals are outward facing. This can be achieved by calling `correctNormals()` which essentially computes the volume of the bounded surface and flips the normals if the volume is negative.

In [3]:
# Compute the normal orientation
mesh.compute_orientation()
mesh.correctNormals()

### Conditioning the protein mesh

In [4]:
# Set selection of all vertices to True so smooth will operate on them.
for v in mesh.vertexIDs:
    v.data().selected = True
# Apply 5 iterations of smoothing
mesh.smooth(max_iter=5, preserve_ridges=True, verbose=True)

Initial Quality: Min Angle = 18.2858, Max Angle = 139.106, # smaller-than-15 = 0, # larger-than-165 = 0
Iteration 1:
Min Angle = 16.9737, Max Angle = 131.859, # smaller-than-15 = 0, # larger-than-165 = 0
Iteration 2:
Min Angle = 17.7987, Max Angle = 130.704, # smaller-than-15 = 0, # larger-than-165 = 0
Iteration 3:
Min Angle = 18.8562, Max Angle = 129.027, # smaller-than-15 = 0, # larger-than-165 = 0
Iteration 4:
Min Angle = 19.0814, Max Angle = 127.025, # smaller-than-15 = 0, # larger-than-165 = 0
Iteration 5:
Min Angle = 19.3171, Max Angle = 124.929, # smaller-than-15 = 0, # larger-than-165 = 0


In [5]:
print(F"The mesh has {mesh.nVertices} vertices, \
{mesh.nEdges} edges, and {mesh.nFaces} faces before coarsening.")

The mesh has 43080 vertices, 129234 edges, and 86156 faces before coarsening.


In [6]:
# Coarsen dense regions of the mesh
mesh.coarse_dense(rate = 2.5, numiter = 5)
# Coarsen flat regions of the mesh
mesh.coarse_flat(rate = 0.016, numiter = 5)

In [7]:
print(F"The mesh has {mesh.nVertices} vertices, \
{mesh.nEdges} edges, and {mesh.nFaces} faces after coarsening.")

The mesh has 20443 vertices, 61323 edges, and 40882 faces after coarsening.


In [8]:
# Smooth the mesh again. Note there's no need to 
mesh.smooth(max_iter = 10, preserve_ridges = True, verbose = True)

Initial Quality: Min Angle = 13.4182, Max Angle = 145.396, # smaller-than-15 = 1, # larger-than-165 = 0
Iteration 1:
Min Angle = 17.898, Max Angle = 133.063, # smaller-than-15 = 0, # larger-than-165 = 0
Iteration 2:
Min Angle = 19.8771, Max Angle = 127.742, # smaller-than-15 = 0, # larger-than-165 = 0
Iteration 3:
Min Angle = 21.9196, Max Angle = 128.362, # smaller-than-15 = 0, # larger-than-165 = 0
Iteration 4:
Min Angle = 22.0856, Max Angle = 128.464, # smaller-than-15 = 0, # larger-than-165 = 0
Iteration 5:
Min Angle = 22.2558, Max Angle = 128.166, # smaller-than-15 = 0, # larger-than-165 = 0
Iteration 6:
Min Angle = 22.6252, Max Angle = 127.548, # smaller-than-15 = 0, # larger-than-165 = 0
Iteration 7:
Min Angle = 23.063, Max Angle = 126.675, # smaller-than-15 = 0, # larger-than-165 = 0
Iteration 8:
Min Angle = 23.5016, Max Angle = 125.613, # smaller-than-15 = 0, # larger-than-165 = 0
Iteration 9:
Min Angle = 23.9163, Max Angle = 124.529, # smaller-than-15 = 0, # larger-than-165 = 

In [9]:
# Set boundary markers of the mesh to 23
for faceID in mesh.faceIDs:
    faceID.data().marker = 23

In [10]:
# Get the root metadata
gInfo = mesh.getRoot()
gInfo.ishole = True    # Don't mesh the inside of 
gInfo.marker = -1

In [11]:
# Center mesh at 0,0,0
center, radius = mesh.getCenterRadius()
mesh.translate(-center)

## Visualizing the mesh with threevis

In [12]:
# Convert mesh to numpy arrays for visualization using threevis
import threevis

protverts, protedges, protfaces = mesh.to_ndarray()
ctx = threevis.Context(width=640, height=480)
ctx.draw_faces(protverts, protfaces)
ctx.draw_edges(protverts, protedges)
# Finally display it
ctx.display()

Renderer(background='#dddddd', camera=PerspectiveCamera(aspect=1.3333333333333333, children=(DirectionalLight(…

## Now let's construct a bounding box to represent the cytosol around the protein

In [13]:
# Generate a surrounding box
box = pygamer.surfacemesh.cube(5)

In [14]:
# Set box boundary markers to 50
for faceID in box.faceIDs:
    faceID.data().marker = 50

In [15]:
# Get and set box metadata
gInfo = box.getRoot()
gInfo.ishole = False
gInfo.marker = 5

In [16]:
# Scale the box by 2 times the radius of the protein mesh
box.scale(radius*2)

In [17]:
boxverts, boxedges, boxfaces = box.to_ndarray()
ctx = threevis.Context(width=640, height=480)
ctx.draw_faces(boxverts, boxfaces)
ctx.draw_edges(boxverts, boxedges)
# Finally display it
ctx.display()

Renderer(background='#dddddd', camera=PerspectiveCamera(aspect=1.3333333333333333, children=(DirectionalLight(…

In [18]:
# Draw the protein and surrounding box together
ctx = threevis.Context(width=640, height=480)
ctx.draw_faces(protverts, protfaces)
ctx.draw_edges(boxverts, boxedges)
ctx.display()

Renderer(background='#dddddd', camera=PerspectiveCamera(aspect=1.3333333333333333, children=(DirectionalLight(…

Write the meshes to file...
`pygamer.writeOFF('2jho.off', mesh)`
`pygamer.writeOFF('box.off', box)`

## Tetrahedralizing

In [19]:
# Construct a list of meshes to pass into TetGen
meshes = [mesh, box]

In [20]:
# Call tetgen to tetrahedralize. The string represents command line arguments passed directly to TetGen.
tetmesh = pygamer.makeTetMesh(meshes, "q1.3/10O8/7AVC")

Number of vertices: 21981
Number of Faces: 43954
Number of Regions: 1
Number of Holes: 1
Region point: Tensor({0}:-11.8786; {1}:16.02; {2}:-0.0702048)
Region point: Tensor({0}:-45.7835; {1}:17.4413; {2}:-17.4413)


In [21]:
# pygamer.writeDolfin('outputTetMesh.xml', tetmesh)

In [ ]:
# Demo the surface by slicing and extracting surface
for vertexID in tetmesh.vertexIDs:
    data = vertexID.data()
    if data[1] > 0:
        tetmesh.removeVertex(vertexID)
        print(tetmesh.nVertices)
print("Done slicing")

# for vertexID in tetmesh.vertexIDs:
#     if not tetmesh.onBoundary(vertexID):
#         print(vertexID)
#         tetmesh.removeVertex(vertexID)

# Convert tetmesh into SurfaceMesh
hull = pygamer.surfacemesh.SurfaceMesh()
for faceID in tetmesh.faceIDs:
    data = faceID.data()
    hull.insertFace(tetmesh.getName(faceID), Face(data.marker, data.selected))
    
for vertexID in tetmesh.vertexIDs:
    data = vertexID.data()
    hull.insertVertex(tetmesh.getName(vertexID), Vertex(data[0], data[1], data[2], 0, false))
    
v, e, f = hull.to_ndarray()

In [ ]:
print(v)

In [ ]:
# Draw the protein and surrounding box together
ctx = threevis.Context(width=640, height=480)
ctx.draw_faces(v, f)
ctx.draw_edges(v, e)
ctx.display()